In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from hud import gym
from hud.utils import stream

from hud.agent import ClaudeAgent

Load a taskset directly from inspect. For now only supports QA environments

In [3]:
from hud.taskset import load_from_inspect
from inspect_evals.gaia.dataset import gaia_dataset

gaia_taskset = load_from_inspect(gaia_dataset())

c:\Users\saecl\Documents\human-union-data\SDK\hud-sdk\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load in WebVoyager_data.jsonl
import jsonl

webv = jsonl.load("WebVoyager_data.jsonl")


In [8]:
import os
import json
from supabase import create_client
from typing import List, Dict, Any

from dotenv import load_dotenv

load_dotenv()

async def upload_tasks_to_supabase(tasks: List[Dict[str, Any]]) -> None:
    """
    Upload a list of tasks to the tasks table in Supabase.
    
    Args:
        tasks: A list of task dictionaries to upload
        
    Returns:
        None
    
    Raises:
        ValueError: If Supabase credentials are not set
        Exception: If there's an error during upload
    """
    # Get Supabase credentials from environment variables
    supabase_url = os.getenv("SUPABASE_URL")
    supabase_key = os.getenv("SUPABASE_KEY")
    
    if not supabase_url or not supabase_key:
        raise ValueError("SUPABASE_URL and SUPABASE_KEY must be set as environment variables")
    
    # Initialize Supabase client
    supabase = create_client(supabase_url, supabase_key)
    
    try:
        # Upload tasks in batches to avoid request size limitations
        batch_size = 100
        for i in range(0, len(tasks), batch_size):
            batch = tasks[i:i+batch_size]
            
            # Insert tasks into the 'tasks' table
            response = supabase.table('tasks').insert(batch).execute()
            
            print(f"Uploaded batch {i//batch_size + 1}/{(len(tasks)-1)//batch_size + 1}: {len(batch)} tasks")
            
        print(f"Successfully uploaded {len(tasks)} tasks to Supabase")
    except Exception as e:
        print(f"Error uploading tasks to Supabase: {str(e)}")
        raise

[autoreload of typing_extensions failed: Traceback (most recent call last):
  File "c:\Users\saecl\Documents\human-union-data\SDK\hud-sdk\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\saecl\Documents\human-union-data\SDK\hud-sdk\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "C:\Users\saecl\AppData\Roaming\uv\python\cpython-3.12.9-windows-x86_64-none\Lib\importlib\__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "c:\Users\saecl\Documents\human-union-data\SDK\hud-sdk\.venv\Lib\site-packages\typing_extensions.py", line 3132, in <module>
    class deprecated:
  Fi

In [10]:
from hud.taskset import TaskSet

In [12]:
gaia_taskset[0].__dict__.keys()


dict_keys(['id', 'prompt', 'setup', 'evaluate', 'gym', 'target', 'choices', 'files', 'metadata', 'config'])

In [25]:
def make_tasks(taskset: TaskSet, evalset_id: list[str]):
    """
    Make a list of tasks from a Taskset.
    """
    return_tasks = []
    for task in taskset.tasks:
        return_tasks.append({
            "prompt": task.prompt,
            "setup": task.setup,
            "evaluate": task.evaluate,
            "evalset_ids": evalset_id,
            "evaluator_id": "aee65f43-c968-4404-85fa-2e17b3faa13e",
            "gym": task.gym,
        })
    return return_tasks




In [26]:
gaia_id = ["fa8375f3-9a48-46f9-a6e8-7b5d98b2e981"]

gaia_tasks = make_tasks(gaia_taskset, gaia_id)


In [27]:
gaia_tasks[0]

{'prompt': "Please answer the question below. You should:\n\n- Return only your answer, which should be a number, or a short phrase with as few words as possible, or a comma separated list of numbers and/or strings.\n- If the answer is a number, return only the number without any units unless specified otherwise.\n- If the answer is a string, don't include articles, and don't use abbreviations (e.g. for states).\n- If the answer is a comma separated list, apply the above rules to each element in the list.\n\n\n\nHere is the question:\n\nA paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?",
 'setup': None,
 'evaluate': ('response_includes', ['egalitarian']),
 'evalset_ids': ['fa8375f3-9a48-46f9-a6e8-7b5d98b2e981'],
 'evaluator_id': 'aee65f43-c96

In [28]:
await upload_tasks_to_supabase(gaia_tasks)

Uploaded batch 1/2: 100 tasks
Uploaded batch 2/2: 65 tasks
Successfully uploaded 165 tasks to Supabase


In [3]:
from hud.taskset import load_taskset
test_taskset = await load_taskset("GAIA")


In [4]:
test_taskset[0]

Task(id='c66a78db-5b2c-4af2-9a78-230ab8fb4f95', prompt="Please answer the question below. You should:\n\n- Return only your answer, which should be a number, or a short phrase with as few words as possible, or a comma separated list of numbers and/or strings.\n- If the answer is a number, return only the number without any units unless specified otherwise.\n- If the answer is a string, don't include articles, and don't use abbreviations (e.g. for states).\n- If the answer is a comma separated list, apply the above rules to each element in the list.\n\n\n\nHere is the question:\n\nA paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?", setup=None, evaluate=('response_includes', ['egalitarian']), gym='qa', target=None, choices=None, files=None, met

1. Try one sample

In [5]:
test = test_taskset[0]
# We'll replace the default gym with a custom gym for computer use models
# If we wanted to test an arbitrary agent, we could just leave the taskset as is with "qa",
# and submit a final response to the evaluate function *see tasks tutorial for more*
# test.gym = "hud-browser" 

from hud.job import create_job
job = await create_job("test-gaia-text")

In [14]:
# Create and set up environment, takes around 20 seconds
env = await gym.make(test, job=job)
# urls = await env.get_urls()

# Stream the live view
# stream(urls["live_url"])

{'id': '052d58bc-a274-4e58-b10d-6c0110dda5a7', 'run_id': 'f4a9c87e-1e91-4372-b2da-e282dae35d50', 'status': 'running', 'metadata': {}, 'start_time': '2025-04-29T07:41:21.756425Z', 'readme': None}


In [18]:
# Define a new agent each time to reset the message history
# Make sure to define the environment variable ANTHROPIC_API_KEY
agent = ClaudeAgent()

# Initial observation
obs, _ = await env.reset()
print(f"Initial observation complete")

# Agent loop
for i in range(50):
    print(f"========= Step {i+1} =========")

    # Use the agent to predict an action
    action, done = await agent.predict(obs)
    print(f"Agent's action (CLA): {action}")
    
    # Step the environment with the action
    obs, reward, terminated, info = await env.step(action)

    if terminated or done:
        break


Initial observation complete
========= Step 1 =========
Agent's action (CLA): [ScreenshotFetch(type='screenshot')]
========= Step 2 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=717, y=398), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 3 =========
Agent's action (CLA): [TypeAction(type='type', text='arXiv AI regulation paper June 2022 figure three axes', selector=None, enter_after=False)]
========= Step 4 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=641, y=469), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 5 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=717, y=398), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 6 =========
Agent's action (CLA): [ResponseAction(type='response', text='I see we\'ve encountered a CAPTCHA. I\'ll need to ask for your help with this situation as I cannot complete CAPTCHAs. \n\nGiven the c

In [12]:
print(await env.evaluate())
await env.close()

{'result': {'logs': 'Response: Based on my research, I need to answer which word from the three axis labels in the 2022 AI regulation paper is used to describe a type of society in a Physics and Society paper from August 11, 2016.\n\nFrom the first search result, I found that the 2022 AI regulation paper has three axes with labels: \n1. Standardization vs. Localization\n2. Utilitarianism vs. Egalitarianism \n3. (The third axis wasn\'t fully visible)\n\nThe most likely answer is "egalitarian" since this is commonly used to describe a type of society, and it appears as one of the axis labels in the 2022 paper.\n\negalitarian includes at least one expected string: [\'egalitarian\']', 'error': None, 'reward': 1.0}, 'stdout': '', 'stderr': ''}


2. Run the full dataset using run_agent

In [31]:
# Make the gym a browser
for task in gaia_taskset:
    task.gym = "hud-browser"

In [32]:
# Smaller subset for testing
gaia_taskset.tasks = gaia_taskset.tasks

In [33]:
from hud.job import run_job

### to see more info:
# import logging
# logging.basicConfig(level=logging.INFO) # or DEBUG

run = await run_job(ClaudeAgent, gaia_taskset, "test-gaia-job", 
                    max_steps_per_task=20)

  1%|----------------------------------------| 36/3300 [1:37<146:48, 22.2 steps/min]HTTP error from HUD SDK: Request failed: Async passthrough error: Request failed: Failed to reset environment: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://www.google.com/", waiting until "load"
 | Status: 500 | Response Text: {"detail":"Failed to reset environment: Page.goto: Timeout 30000ms exceeded.\nCall log:\n  - navigating to \"https://www.google.com/\", waiting until \"load\"\n"} | Response JSON: {'detail': 'Failed to reset environment: Page.goto: Timeout 30000ms exceeded.\nCall log:\n  - navigating to "https://www.google.com/", waiting until "load"\n'} | Headers: {'content-length': '162', 'content-type': 'application/json', 'date': 'Sat, 26 Apr 2025 07:46:09 GMT', 'server': 'railway-edge', 'x-railway-edge': 'railway/us-west2', 'x-railway-request-id': 'QIOa2P0AS1GKfOhg25opYQ_3336177264'} | URL: https://orcstaging.hud.so/hud-gym/api/v2/environments/0a351727-d548-4e67-a

In [34]:
await run.get_analytics()

{'task_count': 171,
 'avg_reward': 0.07692307692307693,
 'success_rate': 0.5847953216374269}

In [35]:
run.errors

[{'task_id': 'task_16',
  'type': 'setup_error',
  'error': 'Request failed: Async passthrough error: Request failed: Failed to reset environment: Page.goto: Timeout 30000ms exceeded.\nCall log:\n  - navigating to "https://www.google.com/", waiting until "load"\n | Status: 500 | Response Text: {"detail":"Failed to reset environment: Page.goto: Timeout 30000ms exceeded.\\nCall log:\\n  - navigating to \\"https://www.google.com/\\", waiting until \\"load\\"\\n"} | Response JSON: {\'detail\': \'Failed to reset environment: Page.goto: Timeout 30000ms exceeded.\\nCall log:\\n  - navigating to "https://www.google.com/", waiting until "load"\\n\'} | Headers: {\'content-length\': \'162\', \'content-type\': \'application/json\', \'date\': \'Sat, 26 Apr 2025 07:46:09 GMT\', \'server\': \'railway-edge\', \'x-railway-edge\': \'railway/us-west2\', \'x-railway-request-id\': \'QIOa2P0AS1GKfOhg25opYQ_3336177264\'} | Status: 500 | Response Text: {"detail":"Async passthrough error: Request failed: Faile

In [ ]:
from hud.job import run_job
from hud.agent import OperatorAgent
### to see more info:
# import logging
# logging.basicConfig(level=logging.INFO) # or DEBUG

run = await run_job(OperatorAgent, gaia_taskset, "test-gaia-operator", 
                    max_steps_per_task=20)

  3%|█---------------------------------------| 99/3300 [15:37<505:05, 6.3 steps/min]No screenshot provided for response to action
No screenshot provided for response to action
No screenshot provided for response to action
No screenshot provided for response to action
No screenshot provided for response to action
No screenshot provided for response to action
  5%|█---------------------------------------| 156/3300 [31:21<631:51, 5.0 steps/min]Network error:  from https://orcstaging.hud.so/hud-gym/api/v2/environments/bee806af-9c8c-4e8c-a01d-60ca7a273c77/invoke, retrying in 2.00 seconds (attempt 1/4)
Network error:  from https://orcstaging.hud.so/hud-gym/api/v2/environments/62c3fca4-0002-44d0-b8a6-f3a7413573d7/invoke, retrying in 2.00 seconds (attempt 1/4)
Network error:  from https://orcstaging.hud.so/hud-gym/api/v2/environments/41798e6d-b2d9-47ee-b495-d7e7fd202573/invoke, retrying in 2.00 seconds (attempt 1/4)
Network error:  from https://orcstaging.hud.so/hud-gym/api/v2/environments/3d2

In [16]:
await run.get_analytics()

DEBUG:hud.job:Fetching trajectories for Job 8c766916-0179-4793-8099-de1c4e7ab3a0 from API...
DEBUG:httpcore.connection:connect_tcp.started host='orcstaging.hud.so' port=443 local_address=None timeout=600.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001EA404FEF90>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001EA041348D0> server_hostname='orcstaging.hud.so' timeout=600.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001EA40488830>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_re

{'task_count': 166, 'avg_reward': 0.0625, 'success_rate': 0.6024096385542169}